Более подробный пример смотри в https://colab.research.google.com/github/csc413-uoft/2021/blob/master/assets/tutorials/tut02_pytorch.ipynb#scrollTo=2xOx3qye69jn

# Обучение модели методом градиентного спуска



In [ ]:
# Подключаем библиотеку, содержащую функции для автоматического вычисления градиента
import autograd.numpy as np
from autograd import grad


: 

In [ ]:
hash

In [2]:
# Функция сигмоиды
def sigmoid(z):
    z = 1/(1+np.exp(-z))
    # декомпозиция функции
    # (a, a')
    # z_4 = -z              # dz_4 = -z' = -1       
    # z_3 = np.exp(z_4)     # dz_3 = np.exp(z_4)' = dz_4 * np.exp(z_4) = -1 * np.exp(-z)
    # z_2 = 1 + z_3         # dz_2 = (1 + z_3)' = dz_3 = -np.exp(-z)
    # z_1 = 1 / z_2         # dz_1 = (1 / z_2)' = (-1 / z_2^2) * dz_2 = (-1 / z_2^2) * (-np.exp(-z)) = np.exp(-z) / (1+np.exp(-z))^2
    return z

In [3]:
# Создадим функцию градиента sigmoid с помощью функции grad
grad_sigmoid = grad(sigmoid)
# Вычислим градиент в точке разными способами
x = 100.0
print(f'y({x})={sigmoid(x)}')
print(f"y'({x})={grad_sigmoid(x)}")
# Сравним значение градиента в точке x
print(sigmoid(x)*(1.0-sigmoid(x)))



y(100.0)=1.0
y'(100.0)=3.720075976020836e-44
0.0


In [4]:
# Функция, вычисляющая выход простого слоя стандартных нейронов
def dense_layer(x, W):
    y = sigmoid(np.dot(x,W))
    return y

In [5]:
# Loss, Error, Cost
# Функция  потерь (ошибки модели) на истинных значениях y_true
def mse(x, W, y_true): # mean absolute error
    y = dense_layer(x, W)
    # error = np.sum(np.abs(y_true-y1))
    error = np.sum(np.square(y_true-y)) # 10 нейронов (2.2, -40.1, 10.8, 0.03, 0.01)
    return error

In [14]:
def solution(l: list) -> list:
    result = []
    j = len(l)-1
    i = 0
    while i<=j:
        if(l[i]**2>l[j]**2):
            result.insert(0, l[i]**2)
            i+=1
        else:
            result.insert(0, l[j]**2)
            j-=1
    return result
    # return sorted([i**2 for i in l])

solution([-3, -2, -1, -0.44, 0, 0.3, 0.5, 4])


[0, 0.09, 0.1936, 0.25, 1, 4, 9, 16]

In [16]:
# [5,7,5,3,2,0], k = 3
def solution(l: list, k: int):
    if(len(l)<k): return max(l)
    result = [max(l[i:i+k]) for i in range(len(l)-k+1)]
    return result

solution([2,76,3,4,6,7,9], 3)


[76, 76, 6, 7, 9]

In [23]:
from collections import deque
def solution(l: list, k: int):
    if(len(l)<k): return max(l)
    q = deque(l[:k], maxlen=k)
    m = max(q)
    result = [m]
    for i in range(k, len(l)):
        first = q.pop()
        if m==first:
            q.append(l[i])
            m = max(q)
        result.append(m)
    return result

solution([2,76,3,4,6,7,9], 3)



[76, 76, 6, 7, 9]

In [ ]:
# [3,4,6,87,9]
def solution(l: list):
    # возврастание
    for i in range(1, len(l)):
        if(l[i]>=l[i-1]):
            continue
        else:
            break
    for i in range(1, len(l)):
        if(l[i]<=l[i-1]):
            continue
        else:
            break

In [6]:
# Вход сети x0
x0 = np.array([1., 1.])
# Матрица весов первого слоя
W1 = np.array([
    [-2., 2.],
    [0.4, -0.9]
])
# выход первого слоя
y = dense_layer(x0, W1)
print(f"y={y}")

y=[0.16798161 0.75026011]


In [7]:
# определяем функцию для вычисления градиента от функции mse
# первый параметр - функция ошибки mse, второй аргумент - номер парараметра mse, по которому считается градиент
mse_grad = grad(mse, 2)

# задаем истинные (желаемые) значения выхода
y_true = np.array([1.0, 0.0])

# вычисляем ошибку модели на истинных значениях
e = mse(x0, W1, y_true)
print(f"error={e}")

# Вычисление функции градиента
grad_value = mse_grad(x0, W1, y_true)
print(f"gradient=\n{grad_value}")


error=1.2551448192484607
gradient=
[ 1.66403677 -1.50052021]


In [8]:
# Уменьшаем значения весов связей на значение градиента
x = np.array([
    [1., 1.],
    [0., 1.],
    [0.5, 0.],
    [0., 0.],
])
y_true = np.array([
    [1.0],
    [0.0],
    [0.0],
    [0.0]
])

# Количество пар выборки (x[i], y_true[i])
pairs_count = x.shape[0]

# Вычислим суммарную ошибку по всем парам обучающей выборки
e = 0
for i in range(pairs_count):
    e += mse(x[i], W1, y_true[i])

# Усреднение ошибки
e /= pairs_count

# e = mse(x0, W1, y_true)
print(f"error={e}")

error=0.5758444614947292


In [9]:
x0 = np.array([2.0, -4.0])
W1 = np.array(
    [
        [1.0, -2.0], 
        [-3.0, -2.0]
    ])
y_true = np.array([0.0, 1.0])
# коэффициент шага обучения
beta = 0.2
for i in range(32):
    y = dense_layer(x0, W1)
    e = mse(x0, W1, y_true)
    grad_value = mse_grad(x0, W1, y_true)
    W1 = W1 - beta * grad_value
    print(f"n={i}, y={y}, error={e}")

n=0, y=[0.99999917 0.98201379], error=1.0003218406934364
n=1, y=[0.99999815 0.98226618], error=1.000310787065863
n=2, y=[0.99999588 0.98251163], error=1.000297606019015
n=3, y=[0.99999083 0.98275041], error=1.0002792165830006
n=4, y=[0.9999796  0.98298279], error=1.000248788219308
n=5, y=[0.9999546  0.98320902], error=1.0001911458271207
n=6, y=[0.99989898 0.98342935], error=1.0000725513666415
n=7, y=[0.99977522 0.983644  ], error=0.9998180013381901
n=8, y=[0.99949996 0.98385319], error=0.9992608932292475
n=9, y=[0.99888827 0.98405712], error=0.9980319524632583
n=10, y=[0.99753136 0.98425599], error=0.9953166844831915
n=11, y=[0.99453326 0.98444998], error=0.9893382007890583
n=12, y=[0.98796661 0.98463927], error=0.976313978233778
n=13, y=[0.97385464 0.98482403], error=0.9486231784245891
n=14, y=[0.94472153 0.98500442], error=0.8927236433230924
n=15, y=[0.88921322 0.98518059], error=0.7909197566983486
n=16, y=[0.79759963 0.98535268], error=0.6363797095417512
n=17, y=[0.67552447 0.985520

In [10]:
x0 = np.array([2.0, -4.0])
W1 = np.array([[4.0, -2.0], [-1.0, -2.0]])
y_true = np.array([0.0, 1.0])
# коэффициент шага обучения
k = 0.2
y = dense_layer(x0, W1)
e = mse(x0, W1, y_true)
grad_value = mse_grad(x0, W1, y_true)
grad_prev = grad_value
p_prev = grad_value
for i in range(64):
    y = dense_layer(x0, W1)
    e = mse(x0, W1, y_true)
    grad_value = mse_grad(x0, W1, y_true)
    beta = np.dot(grad_value, grad_value) / np.dot(grad_prev, grad_prev)
    p = grad_value + beta * p_prev
    W1 = W1 - k * p
    grad_prev = grad_value
    p_prev = p
    print(f"n={i}, y={y}, error={e}")

n=0, y=[0.99999386 0.98201379], error=1.000311215437347
n=1, y=[0.99996957 0.9825151 ], error=1.0002448602138685
n=2, y=[0.9996647  0.98323479], error=0.9996105851734294
n=3, y=[0.99185426 0.98413567], error=0.9840265402702091
n=4, y=[0.70274123 0.9851626 ], error=0.49406539041556746
n=5, y=[0.15697967 0.98582012], error=0.024843684238263438
n=6, y=[0.1262816  0.98600977], error=0.016142769111323625
n=7, y=[0.09975933 0.98628388], error=0.010140055912926953
n=8, y=[0.0796889  0.98659921], error=0.006529901306375704
n=9, y=[0.06481621 0.9869374 ], error=0.0043717722877067025
n=10, y=[0.05365284 0.9872909 ], error=0.003040148479121768
n=11, y=[0.04510124 0.98765571], error=0.002186503667859325
n=12, y=[0.0384156  0.98802905], error=0.0016190617931438759
n=13, y=[0.03309158 0.98840852], error=0.001229415161825882
n=14, y=[0.02878279 0.98879194], error=0.0009540698238984448
n=15, y=[0.02524626 0.98917724], error=0.0007545056848889144
n=16, y=[0.02230786 0.98956245], error=0.000606582951083

In [11]:
# Все тоже самое, но делаем 8 итераций обучения
x0 = np.array([2.0, -4.0])
W1 = np.array([[4.0, -2.0], [-1.0, -2.0]])
y_true = np.array([0.0, 1.0])
# коэффициент шага обучения
k = 0.2
y = dense_layer(x0, W1)
e = mse(x0, W1, y_true)
grad_value = mse_grad(x0, W1, y_true)
grad_prev = grad_value
p_prev = grad_value
for i in range(64):
    y = dense_layer(x0, W1)
    e = mse(x0, W1, y_true)
    grad_value = mse_grad(x0, W1, y_true)
    beta = np.dot(grad_value, grad_value) / np.dot(grad_prev, grad_prev)
    p = grad_value + beta * p_prev
    W1 = W1 - k * p
    grad_prev = grad_value
    p_prev = p
    print(f"y={y}, error={e}")


y=[0.99999386 0.98201379], error=1.000311215437347
y=[0.99996957 0.9825151 ], error=1.0002448602138685
y=[0.9996647  0.98323479], error=0.9996105851734294
y=[0.99185426 0.98413567], error=0.9840265402702091
y=[0.70274123 0.9851626 ], error=0.49406539041556746
y=[0.15697967 0.98582012], error=0.024843684238263438
y=[0.1262816  0.98600977], error=0.016142769111323625
y=[0.09975933 0.98628388], error=0.010140055912926953
y=[0.0796889  0.98659921], error=0.006529901306375704
y=[0.06481621 0.9869374 ], error=0.0043717722877067025
y=[0.05365284 0.9872909 ], error=0.003040148479121768
y=[0.04510124 0.98765571], error=0.002186503667859325
y=[0.0384156  0.98802905], error=0.0016190617931438759
y=[0.03309158 0.98840852], error=0.001229415161825882
y=[0.02878279 0.98879194], error=0.0009540698238984448
y=[0.02524626 0.98917724], error=0.0007545056848889144
y=[0.02230786 0.98956245], error=0.0006065829510832601
y=[0.01984027 0.98994573], error=0.0004947248796761675
y=[0.01774865 0.99032535], error